In [108]:
import pickle
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image


def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

# 최초 손글씨 데이터 로드
(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=False)
# Return (훈련 이미지 개수, 훈련 레이블 크기), (시험 이미지 개수, 시험 레이블 크기)

img = x_train[12321]
label = y_train[32321]
print(label)

print(img.shape)
img = img.reshape(28, 28) # 이미지의 행렬 모양
print(img.shape)
img_show(img)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

def softmax(a):
    c = np.max(a) # 오버 플로우를 감안한 소프트 맥수 함수 구현
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

4
(784,)
(28, 28)
(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [109]:
def get_data():
    (x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=False, one_hot_label=False)
    # normalize = 정규화 유뮤 (식별 능력 개선)
    # 입력 데이터를 정규화하는 것을 전처리한다 라고 부른다
    return x_test, y_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        # pickle 실행 중 특정 객체를 파일로 저장 하는 기능
        network = pickle.load(f)

    return network

def predict(network, X):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    B1, B2, B3 = network["b1"], network["b2"], network["b3"]

    # 입력층
    A1 = np.dot(X, W1) + B1  # 입력 신호 * 가중치
    R1 = sigmoid(A1)         # 활성화 함수 작용

    # 은닉층
    A2 = np.dot(R1, W2) + B2
    R2 = sigmoid(A2)

    # 출력층
    A3 = np.dot(R2, W3) + B3
    y = softmax(A3)          # 분석 결과

    return y


x, t = get_data() # 학습 데이터 로딩
network = init_network() # 신경망 네트워크 생성

batch_size = 100 # 배치 크기
accuracy_cnt = 0 # 정확도 측정 변수
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    print("x_batch.shape :", x_batch.shape)
    print("-------------------------------------------------------")

    y_batch = predict(network, x_batch)
    print("y_batch.size :", y_batch.size)
    print("y_batch.shape :", y_batch.shape)
    print("y_batch :", y_batch[i])
    print("-------------------------------------------------------")
    p = np.argmax(y_batch, axis = 1) # 해당 확률이 가장 높은 원소의 인덱스를 얻는다
    # print(f"p = {p}, t[i] = {t[i]}")

    print("p :", p)
    print("-------------------------------------------------------")
    print("t[i:i + batch_size] :", t[i:i + batch_size])
    print("-------------------------------------------------------")
    print(p == t[i:i + batch_size]) # 배열 p와 배열 t의 값이 같은지 불린 배열 생성
    print("-------------------------------------------------------")

    # print(np.sum(p == t[i:i + batch_size])) # 배열의 True 개수를 더함
    accuracy_cnt += np.sum(p == t[i:i + batch_size])

print("Accuracy :", str(float(accuracy_cnt) / len(x)))

x_batch.shape : (100, 784)
-------------------------------------------------------
y_batch.size : 1000
y_batch.shape : (100, 10)
y_batch : [1.9897243e-07 2.1219858e-08 8.5782312e-06 3.8795356e-06 6.9997825e-09
 2.9318613e-08 2.1244327e-11 4.6365861e-02 1.7664270e-08 2.4632223e-05]
-------------------------------------------------------
p : [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 6 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 7 7 8 9 3 7 4 6 4 3 0 7 0 2 9
 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 4 3 1 4 1 7 6 9]
-------------------------------------------------------
t[i:i + batch_size] : [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3 7 4 6 4 3 0 7 0 2 9
 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 9 3 1 4 1 7 6 9]
-------------------------------------------------------
[ True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True 

/var/folders/yf/mmrj8c9d2zbfn3w8crhhf0_r0000gn/T/ipykernel_47428/2445826353.py:40: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


IndexError: index 100 is out of bounds for axis 0 with size 100

In [ ]:
print(list(range(0, 10)))
x = list(range(0, 10))
print(list(range(2, 11, 3))) # 3을 간격으로 리스트 생성
y = x[1:1 + 2]
print(y)

In [ ]:
x = np.array([
    [0.1, 0.2, 0.2],
    [0.2, 0.3, 0.2],
    [0.1, 0.7, 0.6],
    [0.9, 0.2, 0.8]
])

y = np.argmax(x, axis=1) #
print(y)